# Análise Exploratória - IT Service Ticket Classification

Este notebook apresenta a análise exploratória do dataset de tickets de suporte de TI do Kaggle. O objetivo é compreender a estrutura dos dados, a distribuição das classes e as características dos textos antes de implementar o sistema de classificação.

**Dataset:** IT Service Ticket Classification Dataset  
**Objetivo:** Classificar tickets em 8 categorias e gerar justificativas para cada classificação

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

## 1. Carregamento dos Dados

In [2]:
df = pd.read_csv("../dataset.csv")
print(f"Total de tickets: {len(df):,}")
print(f"Colunas: {list(df.columns)}")
df.head(10)

Total de tickets: 47,837
Colunas: ['Document', 'Topic_group']


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous
5,mail please dear looks blacklisted receiving m...,Miscellaneous
6,prod servers tunneling prod tunneling va la tu...,Hardware
7,access request dear modules report report cost...,HR Support
8,reset passwords for our client and passwords c...,Access
9,direct reports missing time please action repo...,HR Support


O dataset possui duas colunas:
- **Document**: O texto do ticket de suporte
- **Topic_group**: A categoria/classe do ticket

## 2. Qualidade dos Dados

In [3]:
print("Informações do DataFrame:")
print("-" * 40)
df.info()

Informações do DataFrame:
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47837 entries, 0 to 47836
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Document     47837 non-null  object
 1   Topic_group  47837 non-null  object
dtypes: object(2)
memory usage: 747.6+ KB


In [4]:
print("Valores nulos por coluna:")
print(df.isnull().sum())
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

Valores nulos por coluna:
Document       0
Topic_group    0
dtype: int64

Total de valores nulos: 0


In [5]:
print("Valores duplicados:")
duplicates = df.duplicated().sum()
print(f"Tickets duplicados: {duplicates} ({duplicates/len(df)*100:.2f}%)")

Valores duplicados:
Tickets duplicados: 0 (0.00%)


## 3. Distribuição das Classes

In [6]:
class_counts = df["Topic_group"].value_counts().reset_index()
class_counts.columns = ["Classe", "Quantidade"]
class_counts["Percentual"] = (class_counts["Quantidade"] / len(df) * 100).round(2)
class_counts

,Classe,Quantidade,Percentual
0,Hardware,13617,28.47
1,HR Support,10915,22.82
2,Access,7125,14.89
3,Miscellaneous,7060,14.76
4,Storage,2777,5.81
5,Purchase,2464,5.15
6,Internal Project,2119,4.43
7,Administrative rights,1760,3.68


In [7]:
fig = px.bar(
    class_counts,
    x="Classe",
    y="Quantidade",
    color="Classe",
    title="Distribuição de Tickets por Classe",
    text="Quantidade",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_title="Classe",
    yaxis_title="Quantidade de Tickets",
    showlegend=False,
    height=500
)
fig.show()

In [8]:
fig = px.pie(
    class_counts,
    values="Quantidade",
    names="Classe",
    title="Proporção de Tickets por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2,
    hole=0.3
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.update_layout(height=500)
fig.show()

#### Observações sobre a distribuição

- Dataset **desbalanceado**: Hardware e HR Support representam ~51% dos dados
- As classes menores (Administrative rights, Internal Project) têm menos de 5% cada
- Total de **8 classes** distintas para classificação

#### Implicações para o modelo de classificação

1. **Amostragem para avaliação**: Para avaliar em 200 tickets, devemos usar amostragem estratificada para garantir representação de todas as classes.

2. **RAG e exemplos**: O retriever terá mais exemplos de classes majoritárias. Isso pode ser benéfico (mais contexto) ou problemático (viés).

## 4. Análise do Texto dos Tickets

In [10]:
df["text_length"] = df["Document"].str.len()
df["word_count"] = df["Document"].str.split().str.len()

print("Estatísticas da contagem de palavras:")
print(df["word_count"].describe().round(2))

Estatísticas da contagem de palavras:
count    47837.00
mean        43.60
std         56.74
min          2.00
25%         17.00
50%         26.00
75%         46.00
max        981.00
Name: word_count, dtype: float64


In [11]:
fig = px.histogram(
    df,
    x="word_count",
    nbins=50,
    title="Distribuição da Quantidade de Palavras por Ticket",
    color_discrete_sequence=["#66c2a5"]
)
fig.update_layout(
    xaxis_title="Quantidade de Palavras",
    yaxis_title="Frequência",
    height=400
)
fig.show()

In [12]:
# Percentis para entender melhor a distribuição
percentiles = [50, 75, 90, 95, 99]
print("Percentis de palavras por ticket:")
for p in percentiles:
    value = df["word_count"].quantile(p/100)
    print(f"  {p}%: {value:.0f} palavras")

Percentis de palavras por ticket:
  50%: 26 palavras
  75%: 46 palavras
  90%: 91 palavras
  95%: 136 palavras
  99%: 284 palavras


### Discussão: Distribuição do Tamanho dos Textos

A distribuição apresenta assimetria positiva:
- A mediana é significativamente menor que a média, indicando que a maioria dos tickets é curta
- Existem outliers com textos muito longos

In [14]:
fig = px.box(
    df,
    x="Topic_group",
    y="word_count",
    color="Topic_group",
    title="Distribuição de Palavras por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_layout(
    xaxis_title="Classe",
    yaxis_title="Quantidade de Palavras",
    showlegend=False,
    height=500
)
fig.show()

In [15]:
# Estatísticas por classe
stats_by_class = df.groupby("Topic_group")["word_count"].agg(["mean", "median", "std"]).round(1)
stats_by_class = stats_by_class.sort_values("mean", ascending=False)
stats_by_class

,mean,median,std
Topic_group,,,
Hardware,56.3,32.0,75.5
Administrative rights,50.4,30.0,63.9
Miscellaneous,43.0,27.0,52.2
HR Support,37.8,24.0,42.3
Internal Project,37.7,24.0,44.6
Access,35.6,22.0,47.2
Purchase,34.9,30.0,25.9
Storage,34.4,21.0,42.9


## 5. Análise de Palavras Frequentes

In [16]:
all_words = " ".join(df["Document"]).lower().split()
word_freq = Counter(all_words)

print(f"Vocabulário total: {len(word_freq):,} palavras únicas")
print(f"Total de palavras: {len(all_words):,}")

top_words = pd.DataFrame(word_freq.most_common(30), columns=["Palavra", "Frequência"])
top_words

Vocabulário total: 12,327 palavras únicas
Total de palavras: 2,085,566


,Palavra,Frequência
0,please,70212
1,pm,28902
2,hi,28498
3,regards,27212
4,thank,24962
5,for,24456
6,hello,23025
7,you,21828
8,re,21034
9,thanks,19852


In [17]:
fig = px.bar(
    top_words,
    x="Frequência",
    y="Palavra",
    orientation="h",
    title="Top 30 Palavras Mais Frequentes",
    color="Frequência",
    color_continuous_scale="Viridis"
)
fig.update_layout(
    yaxis={"categoryorder": "total ascending"},
    height=700,
    showlegend=False
)
fig.show()

## 6. Palavras Frequentes por Classe

In [18]:
def get_top_words_by_class(df, class_name, n=10):
    class_text = " ".join(df[df["Topic_group"] == class_name]["Document"]).lower().split()
    return Counter(class_text).most_common(n)

classes = df["Topic_group"].unique()
top_words_by_class = []

for cls in classes:
    for word, freq in get_top_words_by_class(df, cls, 10):
        top_words_by_class.append({"Classe": cls, "Palavra": word, "Frequência": freq})

df_top_words = pd.DataFrame(top_words_by_class)

In [19]:
fig = px.bar(
    df_top_words,
    x="Frequência",
    y="Palavra",
    color="Classe",
    facet_col="Classe",
    facet_col_wrap=4,
    orientation="h",
    title="Top 10 Palavras por Classe",
    color_discrete_sequence=px.colors.qualitative.Set2,
    height=800
)
fig.update_layout(showlegend=False)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

## 7. Exemplos de Tickets por Classe

In [20]:
for cls in sorted(df["Topic_group"].unique()):
    count = len(df[df["Topic_group"] == cls])
    pct = count / len(df) * 100
    print(f"\n{'='*80}")
    print(f"CLASSE: {cls.upper()} ({count:,} tickets - {pct:.1f}%)")
    print("="*80)
    samples = df[df["Topic_group"] == cls].sample(2, random_state=42)
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        text = row['Document']
        truncated = text[:300] + "..." if len(text) > 300 else text
        print(f"\n[Exemplo {i}]")
        print(truncated)


CLASSE: ACCESS (7,125 tickets - 14.9%)

[Exemplo 1]
confluence access for wednesday february pm confluence please confluence thanks

[Exemplo 2]
password reset open access dear can you please help with rights for laptop order reset password going expire few days have mention working remote location be around december st also have issues connecting open due rights restrictions you think we could fix somehow best regards

CLASSE: ADMINISTRATIVE RIGHTS (1,760 tickets - 3.7%)

[Exemplo 1]
set up and swap leased for new set up swap leased for

[Exemplo 2]
windows upgrade failed upgrade failed hello please raise ticket for issue below upgrade error thank you kind regards infrastructure manager ext phone sent friday upgrade failed hello sorry bother you can anyone help with upgrade regard developer en sent wednesday upgrade failed hello updating operat...

CLASSE: HR SUPPORT (10,915 tickets - 22.8%)

[Exemplo 1]
access to new pas friday february hi guys please rights involved level thanks di

## 8. Resumo da Análise

In [23]:
print("=" * 60)
print("RESUMO DA ANÁLISE")
print("=" * 60)

print(f"\n[Dataset]")
print(f"  Total de tickets: {len(df):,}")
print(f"  Número de classes: {df['Topic_group'].nunique()}")
print(f"  Sem valores nulos ou duplicados")

print(f"\n[Texto]")
print(f"  Média de palavras: {df['word_count'].mean():.1f}")
print(f"  Mediana de palavras: {df['word_count'].median():.1f}")
print(f"  Máximo de palavras: {df['word_count'].max()}")

print(f"\n[Distribuição das Classes]")
for _, row in class_counts.iterrows():
    print(f"  {row['Classe']}: {row['Quantidade']:,} ({row['Percentual']}%)")

RESUMO DA ANÁLISE

[Dataset]
  Total de tickets: 47,837
  Número de classes: 8
  Sem valores nulos ou duplicados

[Texto]
  Média de palavras: 43.6
  Mediana de palavras: 26.0
  Máximo de palavras: 981

[Distribuição das Classes]
  Hardware: 13,617 (28.47%)
  HR Support: 10,915 (22.82%)
  Access: 7,125 (14.89%)
  Miscellaneous: 7,060 (14.76%)
  Storage: 2,777 (5.81%)
  Purchase: 2,464 (5.15%)
  Internal Project: 2,119 (4.43%)
  Administrative rights: 1,760 (3.68%)


### Conclusões

**Características principais:**
- Dataset com ~48k tickets em 8 classes
- Desbalanceamento entre classes
- Textos curtos em geral (mediana ~26 palavras), com alguns outliers longos
- Textos já pré-processados (lowercase, sem pontuação)

**Implicações para o modelo:**
- Usar amostragem estratificada para conjunto de teste
- Avaliar com F1-score (macro ou weighted) além de accuracy